### Install required packages

In [25]:
!pip install textblob

In [26]:
!pip install --upgrade --user -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Obtaining twint from git+https://github.com/twintproject/twint.git@origin/master#egg=twint
  Updating c:\users\samsung\src\twint clone (to revision origin/master)


  Attempting uninstall: twint
    Found existing installation: twint 2.1.21
    Uninstalling twint-2.1.21:
      Successfully uninstalled twint-2.1.21
  Running setup.py develop for twint


In [152]:
!pip install jupyter-dash

### Import packages

In [1]:
# import necessary libraries
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
from textblob import TextBlob
import re
import time
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

print("Required libraries successfully imported")

Required libraries successfully imported


### Setup Twint

In [2]:
import twint

In [3]:
c = twint.Config()

In [4]:
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio
nest_asyncio.apply()

### Function to Search for Tweets on Crypto

In [5]:
# Searches for tweets based on some keyword
# Returns a dataframe containing the tweets 
def search_tweets(keyword):    
    c.Search = keyword
    c.Verified=True
    c.Lang = "en"
    c.Pandas=True
    c.Limit=100
    c.Pandas_clean=True
    
    twint.run.Search(c)
    
    df_pd_crypto = twint_to_pandas(["date", "username", "tweet", "nlikes"])
    
    return df_pd_crypto

### Function for Saving the Result to Pandas

In [6]:
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

### Function for removing emojis from tweets

In [7]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

### Function for cleaning data 

In [8]:
def clean_data(crypto_df):
    for rows in range(len(crypto_df)):
        crypto_df.tweet[rows]=deEmojify(crypto_df.tweet[rows])  ## DeEmojify
        crypto_df.tweet[rows]=re.sub(r'http\S+', '', crypto_df.tweet[rows])
        crypto_df.tweet[rows]=(re.sub('[^A-Za-z0-9.,'']+', ' ',crypto_df.tweet[rows])) ## Remove special characters
    return crypto_df


### Function to do Sentiment Analysis using Vader

In [9]:
# use VADER to perform sentiment analysis on stored tweets
def vader(crypto):
    crypto_sentiment = SentimentIntensityAnalyzer()
    return crypto_sentiment.polarity_scores(crypto)["compound"]   

### Function to do Sentiment Analysis using TextBlob

In [10]:
def textblob(crypto):
    try:
        return TextBlob(crypto).sentiment
    except:
        return None

### Sentiment Analysis function to call all sentiment analyzers

In [11]:
def sentiment(crypto_df):
    crypto_df["vader"] = crypto_df["tweet"].apply(vader)
    crypto_df['pol_sub'] = crypto_df['tweet'].apply(textblob)
    crypto_df['pol_sub'][0][0]
    crypto_df['textblob'] = crypto_df['pol_sub'].apply(lambda x: x[0])
    crypto_df.drop('pol_sub', inplace=True, axis=1)
    cols = ["vader","textblob"]
    crypto_df[cols] = crypto_df[cols].replace({'0':np.nan, 0:np.nan})
    return crypto_df

### Call the search_tweets function, obtain data, and clean it
### Then call sentiment analysis function

In [12]:
def main():
    keywords=["crypto","bitcoin","ethereum", "DeFi"] #some common keywords to search for
    
    # Create an empty dataframe to store twitter data
    column_names = ["date", "username", "tweet", "nlikes"]
    crypto_df = pd.DataFrame(columns = column_names)
    #Search for tweets corresponding to different keywords
    # Function returns a dataframe for that keyword
    # Combine these dataframes into a single dataframe 
    for keyword in keywords:
        crypto_df=crypto_df.append(search_tweets(keyword), ignore_index=True) #append returned dataframe to overall crypto dataframe
        crypto_df.drop_duplicates(subset ="date", keep = False, inplace = True,ignore_index=True) # remove duplicate tweets
        crypto_df=clean_data(crypto_df)
            
    # Call the different sentiment analysis functions
    crypto_df=sentiment(crypto_df)
    
    return crypto_df
    

In [13]:
#Create a dataframe to store cumulative sentiment scores 
column_names = ["datetime", "vader", "textblob", "vader_positive","textblob_positive"]
sentiment_df = pd.DataFrame(columns = column_names)

In [19]:
# This cell executes the code once every 5 minutes, finally giving a sentiment_df datarframe which contains cumulative sentiment scores for different times
starttime=time.time()
timeout = time.time() + 60*60  # 60 seconds times 60 meaning the script will run for 1 hr
while time.time() <= timeout:
    try:
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        crypto_df=main()
        sentiment_df.loc[len(sentiment_df.index)] = [datetime.now().strftime("%d-%m-%Y %H:%M:%S"), crypto_df['vader'].mean(skipna=True), crypto_df['textblob'].mean(skipna=True),crypto_df.loc[crypto_df['vader'] > 0,'vader'].count()/crypto_df['vader'].count(),crypto_df.loc[crypto_df['textblob'] > 0,'textblob'].count()/crypto_df['textblob'].count()] 
        time.sleep(300) # 5 minute interval between each new execution
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit()


passthrough at  2021-02-21 22:23:14
1363494135240396800 2021-02-21 22:21:48 +0800 <moneycontrolcom> #MCMasterclass⚡️| Here's what @balajis said on banning Crypto in India. Read what more he said here 👇   #Cryptocurrency #Bitcoin  | Watch the Highlights:  https://t.co/VwjqoHYppZ  https://t.co/FUiwsaLNom
1363492654600708097 2021-02-21 22:15:55 +0800 <moneycontrolcom> #MCMasterclass⚡️| @smtgpt, Co-Founder &amp; CEO, @CoinDCX, said that there's a need to regulate #crypto so investor protection is guaranteed. Read more.   #Cryptocurrency #Bitcoin  | Watch the Highlights:  https://t.co/VwjqoHYppZ  https://t.co/zpKyMJF3Lk
1363491574685052928 2021-02-21 22:11:38 +0800 <BarrettSallee> @Bitboy_Crypto Just gonna toss out a theory that it’s F2Pool.
1363491435866161153 2021-02-21 22:11:05 +0800 <FinancialNews> HSBC, JPMorgan among banks dipping into crypto and blockchain — Here’s the list  https://t.co/MjnD2ZdVEK
1363488648172171265 2021-02-21 22:00:00 +0800 <eTorox> "The fact that #Bitcoin inflows

1363494484718350337 2021-02-21 22:23:12 +0800 <prensagrafica> #ElSalvador | En El Zonte se creó una dinámica economía alternativa, #Bitcoin Beach, basada en el uso de criptomonedas. Tiendas, pupuserías, vendedoras de tortillas, minuteros y algunos hoteles las aceptan como medio de pago.  https://t.co/rZAuGOvj3k
1363494443727290371 2021-02-21 22:23:02 +0800 <TorontoStar> It’s easier than ever to invest in bitcoin — but should you?  https://t.co/IZeiMpLLZJ
1363494135240396800 2021-02-21 22:21:48 +0800 <moneycontrolcom> #MCMasterclass⚡️| Here's what @balajis said on banning Crypto in India. Read what more he said here 👇   #Cryptocurrency #Bitcoin  | Watch the Highlights:  https://t.co/VwjqoHYppZ  https://t.co/FUiwsaLNom
1363493696092708871 2021-02-21 22:20:04 +0800 <FinancialNews> Billionaire Jeff Gundlach says bitcoin beats gold as ‘stimulus asset’  https://t.co/DFlAp9Or6q
1363493030733479939 2021-02-21 22:17:25 +0800 <kitun> The #bitcoin laser-eye profile pics are the digital equivalent

1363485002042798081 2021-02-21 21:45:31 +0800 <TRON_VIE> TRON đã phá vỡ thế độc quyền của Ethereum và mọi người đều phát hiện ra rằng chuyển khoản có thể rẻ như vậy. Việc chuyển tiền trên Blockchain về bản chất phải có quy mô lớn, đáng tin cậy và rẻ. Chỉ vì sự độc quyền của ether mà phí đã tăng mạnh.
1363484393327652868 2021-02-21 21:43:06 +0800 <TRON_VIE> Giống như mạng Ethereum và Bitcoin, mạng TRON rất phi tập trung và an toàn.Tổng số tiền TRON U là 14,2 tỷ đô la Mỹ và khối lượng chuyển hàng ngày là 11 tỷ đô la Mỹ. TRON U rẻ, còn Ether U đắt vì thu lợi nhuận độc quyền nên đắt chứ ko phải vì Ether an toàn.
1363473669587140612 2021-02-21 21:00:29 +0800 <ForbesCrypto> Telsa billionaire Elon Musk, whose tweets about ‘joke' meme-based cryptocurrency dogecoin helped push it to a $10 billion valuation, has warned the price of bitcoin and ethereum ‘seem high’: by @BillyBambrough  https://t.co/uMR2m0oSMz
1363469780632690695 2021-02-21 20:45:02 +0800 <TheNationalNews> #ElonMusk said the price

1363493834001416192 2021-02-21 22:20:37 +0800 <OuestFrance> VIDÉO. Mcfly et Carlito relèvent le défi lancé par Emmanuel Macron sur les gestes barrières  https://t.co/XoT2DutQRx
1363492416880324611 2021-02-21 22:14:59 +0800 <PolymathNetwork> Polymath's Greg Murphy will be speaking at "GameStop, DeFi and the Future of Financial Markets”, a Feb 24 WSBA event. Industry experts will analyze recent market events and offer perspectives on what it all means for global markets. Register today!   https://t.co/QERbUpC7J0
1363489293138866178 2021-02-21 22:02:34 +0800 <JMMarchaut> J’étais à 2 doigts d’allumer mon briquet 😂 LA CHANSON DÉFI de @levraimcfly et Carlito suite au message d @EmmanuelMacron sur les gestes barrières. Cliquez si vous en avez marre d’être une vache 🐄 « Je me souviens »  https://t.co/Sz4xZLd0hc via @YouTube  https://t.co/93V73B1mWl
1363485154468130824 2021-02-21 21:46:07 +0800 <CourrierPicard> Les youtubeurs #McFlyetCarlito ont répondu au défi de @EmmanuelMacron , découvrez la

passthrough at  2021-02-21 22:28:30
1363494656970002438 2021-02-21 22:23:53 +0800 <BusinessInsider> Here's why it's actually a good sign that no one is spending their crypto  https://t.co/3YfJYCYY1N
1363494135240396800 2021-02-21 22:21:48 +0800 <moneycontrolcom> #MCMasterclass⚡️| Here's what @balajis said on banning Crypto in India. Read what more he said here 👇   #Cryptocurrency #Bitcoin  | Watch the Highlights:  https://t.co/VwjqoHYppZ  https://t.co/FUiwsaLNom
1363492654600708097 2021-02-21 22:15:55 +0800 <moneycontrolcom> #MCMasterclass⚡️| @smtgpt, Co-Founder &amp; CEO, @CoinDCX, said that there's a need to regulate #crypto so investor protection is guaranteed. Read more.   #Cryptocurrency #Bitcoin  | Watch the Highlights:  https://t.co/VwjqoHYppZ  https://t.co/zpKyMJF3Lk
1363491574685052928 2021-02-21 22:11:38 +0800 <BarrettSallee> @Bitboy_Crypto Just gonna toss out a theory that it’s F2Pool.
1363491435866161153 2021-02-21 22:11:05 +0800 <FinancialNews> HSBC, JPMorgan among banks d

...

1363503746546872320 2021-02-21 23:00:00 +0800 <CanadiensMTL> C'est l’heure du Défi Montréal!  Participez dès maintenant pour courir la chance de remporter votre kit du partisan #MegafanFord 2021!  #GoHabsGo | @FordCanada
1363501234972463105 2021-02-21 22:50:01 +0800 <franceinfo> "Je me souviens" : Mcfly et Carlito dévoilent leur chanson sur les gestes barrières après le défi d’Emmanuel Macron   https://t.co/ubPJF5U8Z2  https://t.co/qjYpfIUJ2b
1363497758657716224 2021-02-21 22:36:12 +0800 <justinsuntron> 🎊Total Value Locked(TVL) in #TRON #DeFi Exceeded $6.2 Billion.  🚀As of 20:30, Feb 21, 2021(HKT), the total value locked in #JustStable, #JustLend, #JustSwap, #JustBTC/ETH/LTC, Total #TRX Frozen, SR Total Votes already exceeded $6.2 Billion.  🙌Thanks to all the users.  https://t.co/8e49guWnSk
1363496241930203141 2021-02-21 22:30:11 +0800 <MiddleEastEyeFr> #InternationalMotherLanguageDay  Alors que la littérature arabe est écrite en fousha, l’apprentissage de la langue peut être un vérita

passthrough at  2021-02-21 23:21:14
1363508961283088384 2021-02-21 23:20:43 +0800 <peterbihr> "This privileged crypto-colonialism almost exclusively benefits white men and it’s happening at a moment when the escalating climate emergency is causing escalating chaos that better deserves our electricity and our attention."    https://t.co/ehrG0qz9er  *This article is on🔥 /2
1363508812645359618 2021-02-21 23:20:08 +0800 <GREGMIKE> This Thursday at 7PM EST my debut NFT collection drops on @niftygateway featuring 55 new works.  As part of the release I've reimagined my MAD CANS series originally released in 2010 with a new set of 50 cans.  #cryptoart #nft #crypto #niftygateway #nftart #digitalart #digital  https://t.co/zCM4GyTaZE
1363507957061754884 2021-02-21 23:16:44 +0800 <chrismattmann> Those lists are focused on munitions and defense but not necessarily on tech - only one I ever really saw on there related to tech was crypto but even that was eased recently   Good model to start from an

1363509030744956928 2021-02-21 23:21:00 +0800 <JamesSurowiecki> @AlanLevinovitz I wrote about this recently, explaining why bitcoin has evolved into an asset - something like digital cold - and doesn't work as a currency:   https://t.co/Zw6Z95cFhO
1363508959102050307 2021-02-21 23:20:43 +0800 <peterbihr> "(..)Bitcoin is deeply colonial, deeply extractive, and deeply damaging to the environment and marginalized people around the world." /1
1363508778826485766 2021-02-21 23:20:00 +0800 <latercera> 📈 @pulso_tw | Bitcoin: La revolución global de las criptomonedas  https://t.co/FcDrKnSAdj  https://t.co/9G87UIZ2LO
1363508760761704449 2021-02-21 23:19:55 +0800 <moneycontrolcom> #MCMasterclass ⚡️| "Continuous dialogues are highly required when the technology is so complex," said Rashmi Deshpande, Partner, @KhaitanCo . Read more.   #Cryptocurrency #Bitcoin  | For Highlights:  https://t.co/51z5dWfXca  https://t.co/HkQoEWnxNu
1363508637977690113 2021-02-21 23:19:26 +0800 <michael_saylor> What hap

1363503935676375040 2021-02-21 23:00:45 +0800 <VanRijmenam> Bitcoin and ethereum prices 'seem high,' says Musk  https://t.co/oTcay1ktvj
1363500015033999364 2021-02-21 22:45:10 +0800 <iProfesional> En la mira: ¿podrá Ethereum destronar a Bitcoin como criptomoneda estrella?  https://t.co/qBNYJTvRNq - Vía @iproup  https://t.co/HKPBHb2vuc
1363496764959952896 2021-02-21 22:32:15 +0800 <SquarespaceHelp> @r_Ethereum At this time it is not possible to accept cryptocurrency payments from products sold on your site. However, we’d be happy to submit this as a feature request on your behalf.
1363485002042798081 2021-02-21 21:45:31 +0800 <TRON_VIE> TRON đã phá vỡ thế độc quyền của Ethereum và mọi người đều phát hiện ra rằng chuyển khoản có thể rẻ như vậy. Việc chuyển tiền trên Blockchain về bản chất phải có quy mô lớn, đáng tin cậy và rẻ. Chỉ vì sự độc quyền của ether mà phí đã tăng mạnh.
1363484393327652868 2021-02-21 21:43:06 +0800 <TRON_VIE> Giống như mạng Ethereum và Bitcoin, mạng TRON rất phi 

1363509101486030848 2021-02-21 23:21:17 +0800 <j_gourault> Un défi lancé par @EmmanuelMacron aux youtubers #McflyEtCarlito pour sensibiliser aux gestes barrières. À aller voir!  https://t.co/MRj05B9m0Y
1363503746546872320 2021-02-21 23:00:00 +0800 <CanadiensMTL> C'est l’heure du Défi Montréal!  Participez dès maintenant pour courir la chance de remporter votre kit du partisan #MegafanFord 2021!  #GoHabsGo | @FordCanada
1363501234972463105 2021-02-21 22:50:01 +0800 <franceinfo> "Je me souviens" : Mcfly et Carlito dévoilent leur chanson sur les gestes barrières après le défi d’Emmanuel Macron   https://t.co/ubPJF5U8Z2  https://t.co/qjYpfIUJ2b
1363497758657716224 2021-02-21 22:36:12 +0800 <justinsuntron> 🎊Total Value Locked(TVL) in #TRON #DeFi Exceeded $6.2 Billion.  🚀As of 20:30, Feb 21, 2021(HKT), the total value locked in #JustStable, #JustLend, #JustSwap, #JustBTC/ETH/LTC, Total #TRX Frozen, SR Total Votes already exceeded $6.2 Billion.  🙌Thanks to all the users.  https://t.co/8e49guW

In [18]:
# Build App
from plotly.subplots import make_subplots

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Market Sentiment on Crypto"),
    dcc.Graph(id='graph'),
    html.Label([
        "NLP Library",
        dcc.Dropdown(
            id='sentiment-dropdown', clearable=False,
            value='vader', options=[
                {'label': 'TextBlob', 'value': 'textblob'},
                {'label': 'Vader', 'value': 'vader'}
            ]),
        html.P("Subplots Height:"),
        dcc.Slider(
            id='slider-width', min=0, max=1, 
            value=0.5, step=0.01)
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input('slider-width',"value"),
    Input('sentiment-dropdown','value')]
)
def update_figure_width(left_width,sentiment):
    #fig = make_subplots(
    #    rows=2, cols=1, column_widths=[left_width, 1 - left_width])
    fig=make_subplots(rows=2,cols=1,row_heights=[left_width, 1 - left_width])
    fig.add_trace(
        go.Scatter(x=sentiment_df['datetime'], y=sentiment_df[sentiment]),
        row=2, col=1)
        
    top_labels = ['Positive', 'Negative'
              ]

    colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)'
              ]

    x_data = [[round(sentiment_df.iloc[-1][sentiment+'_positive']*100,2),round(100*(1-sentiment_df.iloc[-1][sentiment+'_positive']),2)]]

    y_data = ['Percentage wise crypto<br>sentiment based on '+sentiment]
    

    for i in range(0, len(x_data[0])):
        for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Bar(
                x=[xd[i]], y=[yd],
                orientation='h',
                marker=dict(
                    color=colors[i],
                    line=dict(color='rgb(248, 248, 249)', width=1)
                )
            ),row=1,col=1)

    fig.update_layout(
        xaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
            domain=[0.15, 1]
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
        ),
        barmode='stack',
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=120, r=10, t=140, b=80),
        showlegend=False,
    )

    annotations = []

    for yd, xd in zip(y_data, x_data):
        # labeling the y-axis
        annotations.append(dict(xref='paper', yref='y',
                                x=0.14, y=yd,
                                xanchor='right',
                                text=str(yd),
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False, align='right'))
        # labeling the first percentage of each bar (x_axis)
        annotations.append(dict(xref='x', yref='y',
                                x=xd[0] / 2, y=yd,
                                text=str(xd[0]) + '%',
                                font=dict(family='Arial', size=14,
                                          color='rgb(248, 248, 255)'),
                                showarrow=False))
        # labeling the first Likert scale (on the top)
        if yd == y_data[-1]:
            annotations.append(dict(xref='x', yref='paper',
                                    x=xd[0] / 2, y=1.1,
                                    text=top_labels[0],
                                    font=dict(family='Arial', size=14,
                                              color='rgb(67, 67, 67)'),
                                    showarrow=False))
        space = xd[0]
        for i in range(1, len(xd)):
                # labeling the rest of percentages for each bar (x_axis)
                annotations.append(dict(xref='x', yref='y',
                                        x=space + (xd[i]/2), y=yd,
                                        text=str(xd[i]) + '%',
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(248, 248, 255)'),
                                        showarrow=False))
                # labeling the Likert scale
                if yd == y_data[-1]:
                    annotations.append(dict(xref='x', yref='paper',
                                            x=space + (xd[i]/2), y=1.1,
                                            text=top_labels[i],
                                            font=dict(family='Arial', size=14,
                                                      color='rgb(67, 67, 67)'),
                                            showarrow=False))
                space += xd[i]

    fig.update_layout(annotations=annotations)
    
    fig['layout']['xaxis2']['title']='Date and Time'
    fig['layout']['yaxis2']['title']='Average Crypto Sentiment'




# Run app and display result inline in the notebook
    return fig


app.run_server()

Dash app running on http://127.0.0.1:8050/


In [21]:
crypto_df

,date,username,tweet,nlikes,vader,textblob
0,2021-02-21 23:20:08,GREGMIKE,This Thursday at 7PM EST my debut NFT collecti...,1,-0.6037,0.004545
1,2021-02-21 23:16:44,chrismattmann,Those lists are focused on munitions and defen...,0,0.8922,0.180000
2,2021-02-21 23:12:12,TechCabal,Is it legal We spoke to Finance lawyers on ba...,4,0.1280,0.200000
3,2021-02-21 22:41:23,Poloniex,Friends don t let friends invest in crypto,48,0.7351,NaN
4,2021-02-21 22:34:49,DrewsThatDude,"Exclusivity will always be a draw though, VIP ...",2,0.6166,NaN
...,...,...,...,...,...,...
350,2021-02-20 20:05:24,AriDavidPaul,mike 4131 you think it s one and done Defi ha...,2,NaN,NaN
351,2021-02-20 20:00:02,le_Parisien,"Si vous avez 10 millions de vues, je prends u...",174,0.7184,NaN
352,2021-02-20 19:45:00,JVCom,Emmanuel Macron a lanc un d fi Mcfly et Carlit...,1925,0.6705,NaN
353,2021-02-20 19:43:42,valerie_oppelt,D fi pour les gestes barri re covid19 Pour nos...,12,NaN,NaN
